In [1]:
import re
import pandas as pd
from pymongo import MongoClient


# Clean up categorical columns
def clean_list_columns(inpt, regex_pat=r'[a-zA-Z]'):
    if not inpt:
        return []
    
    if not isinstance(inpt, basestring):
        inpt = str(inpt)
    elif not isinstance(inpt, str):
        inpt = inpt.encode('utf-8')
    
    inpt = inpt.split(",")
    temp = []
    for item in inpt:
        temp.append(''.join(re.findall(regex_pat, item)).lower())
    
    inpt = temp

    inpt = ",".join(inpt)
    inpt = inpt.lower()

    return inpt


def clean_numeric_columns(inpt, rexp=re.compile(r'\d+\.\d+')):
    if not inpt:
        return None
    
    if not isinstance(inpt, basestring):
        inpt = str(inpt)
    elif not isinstance(inpt, str):
        inpt = inpt.encode('utf-8')
    
    inpt = rexp.findall(inpt)
    if not inpt:
        return None
    
    return float(inpt[0])

In [2]:
# We do not need these columns as features, at least initially
omit = ['Unnamed: 0', 'access', 'calendar_las_updated', \
            'calendar_last_scraped', 'calendar_updated', 'city', \
            'country', 'country_code', 'description', 'first_review', \
            'host_about', 'host_location', 'host_name', 'host_picture_url', \
            'host_thumbnail_url', 'host_url', 'host_verifications', \
            'house_rules', 'interaction', 'jurisdiction_names', \
            'last_review', 'latitude', 'license', \
            'listing_url', 'longitude', 'market', 'medium_url', \
            'neighborhood_overview', 'neighbourhood', \
            'neighbourhood_group_cleansed', 'notes', 'picture_url', \
            'review_scores_accuracy', 'review_scores_checkin', \
            'review_scores_cleanliness', 'review_scores_communication', \
            'review_scores_location', 'scrape_id', 'smart_location', \
            'space', 'state', 'street', 'summary', 'thumbnail_url', \
            'transit', 'xl_picture_url', 'zipcode']
    
# We need to one_hot encode categorical columns
cat_columns = ['bed_type', 'room_type', \
               'neighbourhood_cleansed', \
               'property_type', 'cancellation_policy', \
               'host_response_time']

list_columns = ['amenities',]

numeric_columns = ['bathrooms', 'bedrooms', 'beds', 'cleaning_fee', \
                       'guests_included', \
                       'host_listings_count', \
                       'host_total_listings_count']

percent_columns = ['host_acceptance_rate', 'host_response_rate']

date_columns = ['host_since', 'last_scraped']

bool_columns = ['host_has_profile_pic', 'host_identity_verified']

In [3]:
print len(omit)
print len(cat_columns)
print len(list_columns)
print len(numeric_columns)
print len(percent_columns)
print len(date_columns)
print len(bool_columns)


47
6
1
7
2
2
2


In [4]:
# Read all the data from MongoDB; 
# We should be batch processing this
client = MongoClient('localhost', 27017)
db = client['airbnb']
coll = db['listings']
cursor = coll.find({})
    
data = []
for sample in cursor:
    for o in omit:
        sample.pop(o, None)
    
    data.append(sample)

data = pd.DataFrame(data)

In [8]:
# Extract dependent variables

review_cols = []
for c in data.columns:
    if 'review' in c: review_cols.append(c)
        
review_cols

[u'number_of_reviews',
 u'review_scores_rating',
 u'review_scores_value',
 u'reviews_per_month']

In [10]:
reviews = pd.DataFrame()
reviews['id'] = data['id']
reviews[review_cols] = data[review_cols].astype(float)

In [11]:
reviews

,id,number_of_reviews,review_scores_rating,review_scores_value,reviews_per_month
0,8388658,0.0,NaN,NaN,NaN
1,1427660,270.0,98.0,10.0,6.62
2,15664146,2.0,100.0,10.0,1.62
3,15285867,5.0,92.0,10.0,1.69
4,16362304,0.0,NaN,NaN,NaN
5,14252951,4.0,100.0,10.0,0.82
6,9904485,0.0,NaN,NaN,NaN
7,2134100,228.0,92.0,9.0,6.39
8,6938818,39.0,90.0,9.0,2.11
9,16064495,0.0,NaN,NaN,NaN


In [12]:
reviews.to_csv('./data/reviews_features.csv', encoding='utf8')

In [51]:
# Clean up categorical columns
cat_dummies = pd.get_dummies(data[cat_columns], prefix=map(lambda x: x.lower(), cat_columns))
cat_cols_df = pd.concat([data['id'], cat_dummies], axis=1)
cat_cols_df

,id,bed_type_Airbed,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,neighbourhood_cleansed_Bayview,...,cancellation_policy_flexible_new,cancellation_policy_moderate,cancellation_policy_no_refunds,cancellation_policy_strict,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60,host_response_time_a few days or more,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour
0,8388658,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1427660,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,15664146,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,15285867,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,16362304,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
5,14252951,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,9904485,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,2134100,0,0,0,0,1,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
8,6938818,0,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
9,16064495,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [71]:
from itertools import chain

def rename_cat_column(cc):    
    cc = cc.lower().split('_')
    cc = map(lambda x: re.findall(r'\w+', x), cc)
    cc = '_'.join(list(chain.from_iterable(cc)))
    return cc

cat_cols_df.rename(columns=lambda x: rename_cat_column(x), inplace=True)

In [72]:
cat_cols_df.to_csv('./data/categorical_features.csv', encoding='utf8')

In [79]:
# Clean up date columns

date_cols_df = pd.DataFrame()
date_cols_df['id'] = data['id']

for dc in date_columns: 
    date_cols_df[dc] = pd.to_datetime(data[dc])

date_cols_df

,id,host_since,last_scraped
0,8388658,2013-11-13,2017-01-01
1,1427660,2012-05-02,2017-01-01
2,15664146,2016-10-24,2017-01-01
3,15285867,2016-08-20,2017-01-01
4,16362304,2015-06-21,2017-01-01
5,14252951,2015-03-30,2017-01-01
6,9904485,2014-03-12,2017-01-01
7,2134100,2013-12-29,2017-01-01
8,6938818,2015-06-21,2017-01-01
9,16064495,2015-01-04,2017-01-01


In [82]:
date_cols_df['days_delta'] = date_cols_df[date_columns[1]] - date_cols_df[date_columns[0]]

In [83]:
date_cols_df.to_csv('./data/date_features.csv', encoding='utf8')

In [4]:
# Clean up list columns
# Convert list columns to one hot encoded columns
list_cols_df = pd.DataFrame()
list_cols_df['id'] = data['id']

for lc in list_columns:
    list_cols_df[lc] = data[lc].apply(lambda x: clean_list_columns(x))

list_cols_df

,id,amenities
0,8388658,"tv,internet,wirelessinternet,kitchen,freeparki..."
1,1427660,"tv,cabletv,internet,wirelessinternet,freeparki..."
2,15664146,"tv,wirelessinternet,airconditioning,kitchen,in..."
3,15285867,"tv,internet,wirelessinternet,kitchen,freeparki..."
4,16362304,"tv,wirelessinternet,kitchen,heating,smokedetec..."
5,14252951,"tv,cabletv,internet,wirelessinternet,kitchen,b..."
6,9904485,"wirelessinternet,kitchen,freeparkingonpremises..."
7,2134100,"tv,cabletv,internet,wirelessinternet,kitchen,g..."
8,6938818,"tv,internet,wirelessinternet,kitchen,heating,f..."
9,16064495,"tv,wirelessinternet,kitchen,freeparkingonpremi..."


In [5]:
list_cols_df['amenities'] = list_cols_df['amenities'].str.lower()

In [27]:
list_cols_df['amenities']

0         tv,internet,wirelessinternet,kitchen,freeparki...
1         tv,cabletv,internet,wirelessinternet,freeparki...
2         tv,wirelessinternet,airconditioning,kitchen,in...
3         tv,internet,wirelessinternet,kitchen,freeparki...
4         tv,wirelessinternet,kitchen,heating,smokedetec...
5         tv,cabletv,internet,wirelessinternet,kitchen,b...
6         wirelessinternet,kitchen,freeparkingonpremises...
7         tv,cabletv,internet,wirelessinternet,kitchen,g...
8         tv,internet,wirelessinternet,kitchen,heating,f...
9         tv,wirelessinternet,kitchen,freeparkingonpremi...
10        tv,cabletv,wirelessinternet,kitchen,heating,fa...
11        tv,internet,wirelessinternet,kitchen,freeparki...
12        tv,cabletv,internet,wirelessinternet,kitchen,p...
13        cabletv,wirelessinternet,kitchen,smokedetector...
14        tv,cabletv,internet,wirelessinternet,kitchen,f...
15        tv,cabletv,internet,wirelessinternet,kitchen,f...
16        tv,wirelessinternet,kitchen,fr

In [30]:
len(pd.unique(list_cols_df.id))

24086

In [11]:
# one_hot_encoude categorical columns
def one_hot_encode(input_dataset, id_col, val_col, prfx, multi_label_splt=','):
    cleaned = input_dataset.set_index(id_col)[val_col].str.split(multi_label_splt, expand=True).stack()
    # .reset_index(level=1, drop=False)
    cleaned = pd.get_dummies(cleaned, prefix=prfx).groupby(id_col, as_index=True).max()
    cleaned[id_col] = cleaned.index
    return cleaned

In [31]:
lc_encoded = one_hot_encode(list_cols_df, 'id', 'amenities', 'amenities', ',')
list_cols_df = list_cols_df.join(lc_encoded.set_index('id'), on='id')
list_cols_df

,id,amenities,amenities_,amenities_accessibleheightbed,amenities_accessibleheighttoilet,amenities_airconditioning,amenities_amazonecho,amenities_babybath,amenities_babymonitor,amenities_babysitterrecommendations,...,amenities_wideclearancetoshower,amenities_wideclearancetoshowerandtoilet,amenities_wideclearancetoshowertoilet,amenities_widedoorway,amenities_wideentryway,amenities_widehallwayclearance,amenities_wifi,amenities_windowguards,amenities_winecooler,amenities_wirelessinternet
0,8388658,"tv,internet,wirelessinternet,kitchen,freeparki...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1427660,"tv,cabletv,internet,wirelessinternet,freeparki...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2,15664146,"tv,wirelessinternet,airconditioning,kitchen,in...",0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,15285867,"tv,internet,wirelessinternet,kitchen,freeparki...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,16362304,"tv,wirelessinternet,kitchen,heating,smokedetec...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
5,14252951,"tv,cabletv,internet,wirelessinternet,kitchen,b...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,9904485,"wirelessinternet,kitchen,freeparkingonpremises...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,2134100,"tv,cabletv,internet,wirelessinternet,kitchen,g...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
8,6938818,"tv,internet,wirelessinternet,kitchen,heating,f...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
9,16064495,"tv,wirelessinternet,kitchen,freeparkingonpremi...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [32]:
list_cols_df.to_csv('./data/amenities_features.csv', encoding='utf8')

In [41]:
# Cleanup bool columns
bool_cols_df = pd.DataFrame()
bool_cols_df['id'] = data['id']

for bc in bool_columns:
    bool_cols_df[bc] = data[bc].astype(bool).astype(int)

bool_cols_df

,id,host_has_profile_pic,host_identity_verified
0,8388658,1,1
1,1427660,1,1
2,15664146,1,1
3,15285867,1,1
4,16362304,1,1
5,14252951,1,1
6,9904485,1,1
7,2134100,1,1
8,6938818,1,1
9,16064495,1,1


In [42]:
bool_cols_df.to_csv('./data/bool_features.csv', encoding='utf8')

In [11]:
# Clean up Percent columns
percent_cols_df = pd.DataFrame()
percent_cols_df['id'] = data['id']
for pc in percent_columns:
    percent_cols_df[pc] = data[pc].str.rstrip('%').astype('float') / 100.0

percent_cols_df

# for d in data['host_response_rate'].str.rstrip('%').astype('float') / 100.0:
#     if pd.notna(d):
#         print d


,id,host_acceptance_rate,host_response_rate
0,8388658,NaN,NaN
1,1427660,NaN,1.00
2,15664146,NaN,1.00
3,15285867,NaN,1.00
4,16362304,NaN,1.00
5,14252951,NaN,0.67
6,9904485,NaN,NaN
7,2134100,NaN,0.95
8,6938818,NaN,1.00
9,16064495,NaN,0.80


In [57]:
percent_cols_df.shape

(243721, 3)

In [ ]:
# percent_cols_df.to_csv('./data/percent_cols.csv', encoding='utf-8')

In [31]:
# Clean up Numeric columns
import numpy as np 

numeric_cols_df = pd.DataFrame()
numeric_cols_df['id'] = data['id']

for nc in numeric_columns:
    numeric_cols_df[nc] = data[nc].fillna(np.nan)\
    .astype('str').str.replace(',', '')\
    .str.replace('%', '')\
    .str.replace('$', '')\
    .astype('float')

numeric_cols_df.shape

(243721, 8)

In [56]:
numeric_cols_df.shape

(243721, 8)

In [ ]:
# numeric_cols_df.to_csv('./data/numeric_feat.csv', encoding='utf-8')

In [17]:
# joined_num_perc = numeric_cols_df.merge(percent_cols_df, how='inner', left_on='id', right_on='id')

In [81]:
joined_df = percent_cols_df.drop('id', axis=1)
joined_df = pd.concat([numeric_cols_df, joined_df], axis=1)

In [82]:
joined_df.shape

(243721, 10)

In [86]:
joined_df.to_csv('./data/numeric_features.csv', encoding='utf-8')

In [ ]:
# Remove numeric dataframe copies from memory

percent_cols_df = None
numeric_cols_df = None
numeric = None

In [ ]:
# data = data.drop('_id', axis=1)

In [ ]:
# data = data.drop(cat_columns, axis=1)

In [ ]:
# data.columns

In [ ]:
# data.to_csv('./data/review_features.csv', encoding='utf-8')

In [ ]:
# payload = json.loads(data.to_json(orient='records', force_ascii=False))
# coll.insert_many(payload)